In [ ]:

import os
import sys
import shutil
import subprocess
import time
import requests

# Setup
path = os.getcwd()
wfile = os.path.join(path, "working.m3u")
temp_dir = os.path.join(path, "temp")
null_file = os.path.join(path, "null")

# Colors
class Colors:
    CYAN = '\033[0;36m'
    GREEN = '\033[92m'
    RED = '\033[91m'
    YELLOW = '\033[93m'
    ORANGE = '\033[38;5;166m'
    END = '\033[0m'

def print_logo():
    print(f"{Colors.GREEN}IPTV-Check Tool 1.0{Colors.END}")
    print(f"{Colors.YELLOW}{'-' * 37}{Colors.END}")
    print(f"{Colors.CYAN}http://github.com/peterpt{Colors.END}")
    print(f"{Colors.YELLOW}{'-' * 37}{Colors.END}\n")

def cleanup_temp():
    if not os.path.exists(temp_dir):
        os.makedirs(temp_dir)
    else:
        for file in os.listdir(temp_dir):
            file_path = os.path.join(temp_dir, file)
            if os.path.isfile(file_path):
                os.remove(file_path)

def download_file(url, dest):
    try:
        r = requests.get(url, timeout=10)
        if r.status_code == 200:
            with open(dest, 'wb') as f:
                f.write(r.content)
            return True
    except Exception:
        pass
    return False

def get_file_size(file_path):
    return os.path.getsize(file_path) if os.path.exists(file_path) else 0

def writefile(line_before_url, url):
    with open(wfile, 'a') as f:
        f.write(f"{line_before_url}\n{url}\n\n")

def parse_links(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()
    links = [line.strip() for line in lines if line.strip().startswith(("http", "rtmp", "HTTP"))]
    return list(dict.fromkeys(links))  # remove duplicates while preserving order

def test_stream(url, index, total, line_before_url):
    temp_stream = os.path.join(temp_dir, "stream")
    pid = subprocess.Popen(["wget", "-q", url, "-O", temp_stream])
    time.sleep(4)
    pid.terminate()

    if not os.path.exists(temp_stream):
        print(f"{Colors.YELLOW}Error reading captured file{Colors.END}")
        return

    if get_file_size(temp_stream) > 100:
        print(f"{Colors.GREEN}Link:{Colors.YELLOW} {index}{Colors.GREEN} of {Colors.YELLOW}{total}{Colors.GREEN} is {Colors.GREEN}ON{Colors.END}")
        writefile(line_before_url, url)
    else:
        print(f"{Colors.GREEN}Link:{Colors.YELLOW} {index}{Colors.GREEN} of {Colors.YELLOW}{total}{Colors.GREEN} is {Colors.RED}OFF{Colors.END}")
    os.remove(temp_stream)

def process_m3u(input_file):
    shutil.copy(input_file, os.path.join(temp_dir, "1"))
    lines = open(os.path.join(temp_dir, "1")).readlines()
    links = []
    line_map = {}
    for i, line in enumerate(lines):
        if line.strip().startswith(("http", "rtmp", "HTTP")):
            links.append(line.strip())
            line_map[line.strip()] = lines[i - 1].strip() if i > 0 else "#EXTINF:-1"

    with open(wfile, 'w') as f:
        f.write("#EXTM3U\n\n")

    for idx, url in enumerate(links, start=1):
        test_stream(url, idx, len(links), line_map.get(url, "#EXTINF:-1"))

    print(f"\n{Colors.GREEN}Job Finished{Colors.END}")
    print(f"{Colors.YELLOW}Your IPTV list was updated in:{Colors.END}")
    print(f"{Colors.ORANGE}{wfile}{Colors.END}")

def save_working_file(output_path):
    if os.path.exists(wfile):
        shutil.copyfile(wfile, output_path)
        print(f"{Colors.GREEN}Playlist also saved to:{Colors.END} {Colors.CYAN}{output_path}{Colors.END}")
    else:
        print(f"{Colors.RED}No working file to save!{Colors.END}")

def main():
    if len(sys.argv) < 2:
        print_logo()
        print(f"{Colors.ORANGE}Example for remote list to check:{Colors.END}")
        print(f"{Colors.GREEN}python {sys.argv[0]} http://someurl/somelist.m3u [output.m3u]{Colors.END}")
        print(f"{Colors.ORANGE}Example for local list to check:{Colors.END}")
        print(f"{Colors.GREEN}python {sys.argv[0]} /root/mylist.m3u [output.m3u]{Colors.END}")
        sys.exit(1)

    input_arg = sys.argv[1]
    output_file = sys.argv[2] if len(sys.argv) > 2 else None

    cleanup_temp()

    if input_arg.startswith("http"):
        success = download_file(input_arg, os.path.join(temp_dir, "1"))
        if not success or get_file_size(os.path.join(temp_dir, "1")) <= 10:
            print(f"{Colors.YELLOW}Remote file is invalid or too small.{Colors.END}")
            sys.exit(1)
        process_m3u(os.path.join(temp_dir, "1"))
    else:
        if not os.path.isfile(input_arg):
            print(f"{Colors.YELLOW}Local file not found:{Colors.END} {Colors.GREEN}{input_arg}{Colors.END}")
            sys.exit(1)
        if get_file_size(input_arg) <= 10:
            print(f"{Colors.YELLOW}Local file is too small to be a valid M3U playlist.{Colors.END}")
            sys.exit(1)
        process_m3u(input_arg)

    if output_file:
        save_working_file(output_file)
        process_m3u(input_arg)

# if name == "main":
#     main()

In [5]:
!python iptv_check.py sports.m3u checked_list.m3u

python: can't open file 'd:\\M3U_python\\iptv_check.py': [Errno 2] No such file or directory
